In [1]:
import gensim
import sklearn
import numpy as np
import pandas as pd
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from random import shuffle
import pandas as pd
from konlpy.tag import Okt
print('gensim  version = {}'.format(gensim.__version__))
print('sklearn version = {}'.format(sklearn.__version__))

Duplicate key in file '/Users/seungheondoh/.matplotlib/matplotlibrc' line #2.


gensim  version = 3.8.1
sklearn version = 0.21.2


In [2]:
dataset_path = 'kor_data.csv'
remove_item = ('','Conjunction','Verb','Determiner', 'Exclamation','Josa','Punctuation','Suffix', 'Foreign', 'Number','Adverb')
pos_tagger = Okt()

In [3]:
df = pd.read_csv('./kor_data.csv')

In [4]:
def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

def main(dataset_path, remove_item):
    df = pd.read_csv(dataset_path)
    docs = list(df['디스크립션'])
    train_docs = [tokenize(row) for row in docs]
    trained_docs_list = []
    for item in train_docs:
        dummy = []
        for indi in item:
            if (indi.split('/')[1] not in remove_item):
                dummy.append(indi.split('/')[0])
        trained_docs_list.append(dummy)
    
    df['text'] = trained_docs_list
    return df
df = main(dataset_path, remove_item)

In [5]:
zipdata = zip(df['프로젝트명'],df['text'])

In [6]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in zipdata]

In [7]:
model = gensim.models.Doc2Vec(vector_size=100, window=10, min_count=1, workers=8, alpha=0.025, min_alpha=0.015, 
                              epochs=500)
#sample=1e-4, negative=5
#shuffling is better (ot needed at each trianing epoch
shuffle(tagged_data)
#Build vocabulary from a sequence of sentences 
model.build_vocab(tagged_data)
#Update the model’s neural weights from a sequence of sentences
model.train(tagged_data, epochs=model.epochs, total_examples=model.corpus_count)

In [41]:
import pickle
# with open('model.pkl', 'wb') as f:
#     pickle.dump(model, f)
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

# Json

In [42]:
artist = list(df['작가명'].unique())
artist

['김정태', '박승순', '양아치', '이장원', '조호영', '최진훈', '최병일', '현박']

In [43]:
def artist_vec(artist, df, dv, wv, idx_to_doc, idx_to_vocab, tagged_data):
    project = pd.DataFrame(dv, index=idx_to_doc)
    word = pd.DataFrame(wv, index=idx_to_vocab)
    allvector = []
    for i in list(df[df['작가명'] == artist]['프로젝트명']):
        allvector.append(project.loc[str(i)])
#         for word_vec, name in tagged_data:
#             if i == name[0]:
#                 for j in word_vec:
#                     allvector.append(word.loc[str(j)])
    np_all = np.array(allvector)
    artist_vec = np.mean(np_all, axis=0)
    return artist_vec

In [44]:
wv = model.wv.vectors
dv = model.docvecs.vectors_docs

idx_to_doc = model.docvecs.index2entity
idx_to_vocab = model.wv.index2word

In [45]:
artist_100 = []
for i in artist:
    artist_100.append(artist_vec(i, df, dv, wv, idx_to_doc, idx_to_vocab, tagged_data))

In [46]:
av = np.array(artist_100)

In [47]:
# Project =, word, Key충돌

new_doc = []
for i in idx_to_doc:
    projectName = 'P_'+i
    new_doc.append(projectName)
    
new_doc[0]

'P_Sally'

In [48]:
def makeUniqe(doc,docType='Project'):
    new_doc = []
    for i in doc:
        new_name =docType + '_' + i
        new_doc.append(new_name)
    return new_doc

In [49]:
new_doc = makeUniqe(idx_to_doc,'Project')
new_art = makeUniqe(artist,'Artist')

In [50]:
artist_index = len(new_art)
doc_index = len(idx_to_doc)
word_index = len(idx_to_vocab)
entity_list = new_art + new_doc + idx_to_vocab
vec100 = np.vstack([av, dv, wv])
df_total = pd.DataFrame(vec100, index=entity_list)

In [51]:
from sklearn.metrics.pairwise import cosine_similarity
temp = cosine_similarity(df_total)
df_similarity = pd.DataFrame(temp, columns=entity_list)

In [52]:
def get_similarity(df_similarity, query, new_art, new_doc, idx_to_vocab, top_n = 5):
    artist_index = len(new_art)
    doc_index = len(new_doc)
    word_index = len(idx_to_vocab)

    artist_top5 = df_similarity[query].iloc[:artist_index].sort_values(ascending=False)[1:top_n+1]
    artist_result = []
    for idx,sim in zip(artist_top5.index,artist_top5.values):
        documents = {}
        documents['key'] = new_art[idx]
        documents['similarity'] = sim
        artist_result.append(documents)
        
    doc_top5 = df_similarity[query].iloc[artist_index:artist_index+doc_index].sort_values(ascending=False)[1:top_n+1]
    doc_result = []
    for idx,sim in zip(doc_top5.index, doc_top5.values):
        documents = {}
        documents['key'] = new_doc[idx-artist_index]
        documents['similarity'] = sim
        doc_result.append(documents)
        
    word_top5 = df_similarity[query].iloc[artist_index+doc_index:].sort_values(ascending=False)[1:top_n+1]
    word_result = []
    for idx,sim in zip(word_top5.index, word_top5.values):
        documents = {}
        documents['key'] = idx_to_vocab[idx-artist_index-doc_index]
        documents['similarity'] = sim
        word_result.append(documents)
    
    return artist_result, doc_result, word_result

In [53]:
new_art[2]

'Artist_양아치'

In [118]:
get_similarity(df_similarity, 'Project_비 독창적 비 적층식 액상 조형장치', new_art, new_doc, idx_to_vocab)

([{'key': 'Artist_최진훈', 'similarity': 0.68887335},
  {'key': 'Artist_조호영', 'similarity': 0.68617564},
  {'key': 'Artist_김정태', 'similarity': 0.6452769},
  {'key': 'Artist_이장원', 'similarity': 0.6113128},
  {'key': 'Artist_박승순', 'similarity': 0.60041225}],
 [{'key': 'Project_Screen Screen Screen', 'similarity': 0.662458},
  {'key': 'Project_ZER01NE MAGAZINE', 'similarity': 0.6278928},
  {'key': 'Project_엠알랩', 'similarity': 0.6218226},
  {'key': 'Project_어디가 프로젝트', 'similarity': 0.6168661},
  {'key': 'Project_아름다운 비행', 'similarity': 0.58710253}],
 [{'key': '조형', 'similarity': 0.74239916},
  {'key': '미리', 'similarity': 0.74176913},
  {'key': '자살', 'similarity': 0.7363745},
  {'key': '어렵다', 'similarity': 0.71748704},
  {'key': '매혹', 'similarity': 0.7170589}])

In [55]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def embedding(x, method='tsne', perplexity=10):
    if method == 'pca':
        trainer = PCA(n_components=2)
    else:
        trainer = TSNE(n_components=2, perplexity=perplexity)
    return trainer.fit_transform(x)

In [56]:
av.shape[0],wv.shape[0],dv.shape[0]

(8, 1124, 22)

In [57]:
z_merge = embedding(np.vstack([wv, dv]), 'tsne')
z_wv = z_merge[:wv.shape[0]]
z_dv = z_merge[wv.shape[0]:]

In [58]:
artist

['김정태', '박승순', '양아치', '이장원', '조호영', '최진훈', '최병일', '현박']

In [59]:
z_av = []
for i in artist:
    z_av.append(artist_vec(i, df, z_dv, z_wv, idx_to_doc, idx_to_vocab, tagged_data))

In [60]:
z_av = np.array(z_av)

In [61]:
len(z_av), len(z_wv), len(z_dv)

(8, 1124, 22)

In [62]:
doc_index = len(idx_to_doc)
word_index = len(idx_to_vocab)

In [63]:
group_a = []
for i in range(len(artist)):
    group_a.append('artist')
group_w = []
for i in range(len(idx_to_vocab)):
    group_w.append('word')
group_p = []
for i in range(len(new_doc)):
    group_p.append('project')

In [64]:
col = ['key','id','x','y','Group']
artist_data = {'key':new_art,'id':artist, 'x':z_av.T[0],'y':z_av.T[1],'Group': group_a}
doc_data = {'key':new_doc,'id':idx_to_doc,'x':z_dv.T[0],'y':z_dv.T[1],'Group': group_p}
word_data = {'key':idx_to_vocab,'id':idx_to_vocab,'x':z_wv.T[0],'y':z_wv.T[1],'Group': group_w}

In [65]:
a = pd.DataFrame(artist_data, columns=col)
b = pd.DataFrame(word_data, columns=col)
c = pd.DataFrame(doc_data, columns=col)

In [66]:
total = pd.concat([a,b,c])
total.head()

,key,id,x,y,Group
0,Artist_김정태,김정태,-25.856247,4.081685,artist
1,Artist_박승순,박승순,12.920586,6.756471,artist
2,Artist_양아치,양아치,20.153425,-16.116323,artist
3,Artist_이장원,이장원,35.738045,-15.674254,artist
4,Artist_조호영,조호영,22.023046,13.988201,artist


In [69]:
def get_artist_db(df, query):
    query = query.split('_')[1]
    project = list(df[df['작가명']== query]['프로젝트명'])
    inside = list(df[df['작가명']==query]['내부파트너'].dropna().unique())
    outside = list(df[df['작가명']==query]['외부파트너'].dropna().unique())
    media = list(df[df['작가명']==query]['매체'].dropna().unique())
    
    key_list = []
    keyword = ""
    for i in list(df[df['작가명']==query]['키워드'])[0]:
        if i != '#':
            keyword = keyword + i
        elif len(keyword) > 1:
            key_list.append(keyword.strip())
            keyword = ""
    
    documents = {}
    documents['id'] = query
    documents['project'] = project
    documents['key_list'] = key_list
    documents['media'] = media
    documents['inside'] = inside
    documents['outside'] = outside
    return documents

In [70]:
get_artist_db(df, 'Artist_박승순')

{'id': '박승순',
 'project': ['모멘텀',
  '인터미디어 모빌리티 R&D',
  '창의R&d',
  'NeuroScape for Mobility(N4M)',
  'Music for Vehicles'],
 'key_list': ['아쿠아포닉스', '물의 연주', '딥러닝 알고리즘', '이미지와 사운드'],
 'media': ['음악/사운드 설치',
  '아카이브 및 프레젠테이션, 비디오, 사운드, 키노트 프리젠테이션',
  '혼합매체, 가변설치',
  '퍼포먼스, 8채널 사운드, 8채널 비디오, 가변설치'],
 'inside': ['박승순, 전형산', '창의생태연구모임(박승순, 우아름, 임서진)', '박성민, 박승순'],
 'outside': ['서지혜, 도승헌']}

In [119]:
import collections


def get_project_db(df, query):
    sim_query = query
    query = query.split('_')[1]
    artist = df[df['프로젝트명']== query]['작가명'].values[0]
    descrtibe = list(df[df['프로젝트명']== query]['디스크립션'])[0]
    inside = list(df[df['프로젝트명']== query]['내부파트너'])
    outside = list(df[df['프로젝트명']== query]['외부파트너'])
    keyword_list = df[df['프로젝트명']== query]['text'].values[0]
    keyword = collections.Counter(keyword_list).most_common(5)
    year = df[df['프로젝트명']== query]['년도'].values[0]
    sim_art,sim_doc,sim_word = get_similarity(df_similarity, sim_query , new_art, new_doc, idx_to_vocab)
    
    documents = {}
    documents['id'] = query
    documents['artist'] = artist
    documents['descrtibe'] = descrtibe
    documents['inside'] = inside
    documents['outside'] = outside
    documents['keyword'] = keyword
    documents['year'] = year
    documents['sim_doc'] = sim_doc
    documents['sim_word'] = sim_word
    
    return documents

In [120]:
def get_keyword_db(df, query):
    use_artist = []
    use_project = []

    for idx,i in enumerate(df['text']):
        if query in i:
            use_artist.append(df.iloc[idx]['작가명'])
            use_project.append(df.iloc[idx]['프로젝트명'])
    
    sim_art,sim_doc,sim_word = get_similarity(df_similarity, query , new_art, new_doc, idx_to_vocab)
    
    documents = {}
    documents['id'] = query
    documents['use_artist'] = list(set(use_artist))
    documents['use_project'] = list(set(use_project))
    documents['sim_art'] = sim_art
    documents['sim_doc'] = sim_doc
    documents['sim_word'] = sim_word
    
    return documents

In [122]:
attribute_a = []
for i in new_art:
    attribute_a.append(get_artist_db(df, i))
    
attribute_d = []
for i in new_doc:
    attribute_d.append(get_project_db(df, i))
    
attribute_w = []
for i in idx_to_vocab:
    attribute_w.append(get_keyword_db(df, i))

In [126]:
attribute = attribute_a + attribute_d + attribute_w

In [127]:
total['attritube'] = attribute

In [128]:
total.head()

,key,id,x,y,Group,attritube
0,Artist_김정태,김정태,-25.856247,4.081685,artist,"{'id': '김정태', 'project': ['팀잇', '어디가 프로젝트'], '..."
1,Artist_박승순,박승순,12.920586,6.756471,artist,"{'id': '박승순', 'project': ['모멘텀', '인터미디어 모빌리티 R..."
2,Artist_양아치,양아치,20.153425,-16.116323,artist,"{'id': '양아치', 'project': ['Screen Screen Scree..."
3,Artist_이장원,이장원,35.738045,-15.674254,artist,"{'id': '이장원', 'project': ['아르키메데스 구멍 - 149,597..."
4,Artist_조호영,조호영,22.023046,13.988201,artist,"{'id': '조호영', 'project': ['랜드스케이프데이터프로젝트', '엠알..."


In [139]:
total_dic = total.to_dict('record')

In [148]:
total_dic[1],total_dic[30],total_dic[100] 

({'key': 'Artist_박승순',
  'id': '박승순',
  'x': 12.920585632324219,
  'y': 6.756471157073975,
  'Group': 'artist',
  'attritube': {'id': '박승순',
   'project': ['모멘텀',
    '인터미디어 모빌리티 R&D',
    '창의R&d',
    'NeuroScape for Mobility(N4M)',
    'Music for Vehicles'],
   'key_list': ['아쿠아포닉스', '물의 연주', '딥러닝 알고리즘', '이미지와 사운드'],
   'media': ['음악/사운드 설치',
    '아카이브 및 프레젠테이션, 비디오, 사운드, 키노트 프리젠테이션',
    '혼합매체, 가변설치',
    '퍼포먼스, 8채널 사운드, 8채널 비디오, 가변설치'],
   'inside': ['박승순, 전형산', '창의생태연구모임(박승순, 우아름, 임서진)', '박성민, 박승순'],
   'outside': ['서지혜, 도승헌']}},
 {'key': '크리에이터',
  'id': '크리에이터',
  'x': -60.22866439819336,
  'y': 44.926177978515625,
  'Group': 'word',
  'attritube': {'id': '있다',
   'use_artist': ['양아치', '최진훈', '박승순', '김정태', '조호영', '최병일', '이장원', '현박'],
   'use_project': ['아름다운 비행',
    'Music for Vehicles',
    'WILSON',
    '소프트로봇 연구',
    '아르키메데스 구멍 - 149,597,870,700, 2018',
    '랜드스케이프데이터프로젝트',
    '하얀 방 ',
    'Screen; Less is More',
    '비 독창적 비 적층식 액상 조형장치',
    '창의R&d',
    '인터미디어 모빌리티 R&D'

In [149]:
import json
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

# Your codes .... 
with open('total_dic.json', 'w', encoding='UTF-8') as outfile:
    outfile.write(json.dumps(total_dic, cls=NpEncoder, ensure_ascii=False))

In [150]:
# prepare visualization
import numpy as np
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.plotting import output_notebook, output_file, reset_output

def initialize_figure(title):
    if title is None:
        title = 'Untitled'

    TOOLTIPS = [
        ("(x,y)", "($x, $y)"),
        ("entity", "@desc"),
    ]

    p = figure(title=title, tooltips=TOOLTIPS)
    p.grid.grid_line_color = None
    p.background_fill_color = "black"
    p.width = 600
    p.height = 600
    return p

def mtext(p, x, y, text, text_color, text_font_size, text_alpha):
    p.text(x, y, text=[text], text_color=text_color, text_align="center",
           text_font_size=text_font_size, text_alpha=text_alpha)

def annotation(p, coordinates, idx_to_text, text_shift=0.05,
    text_font_size='15pt', text_color="firebrick", text_alpha=1.0):

    for idx, text in enumerate(idx_to_text):
        if not text:
            continue
        x_ = coordinates[idx,0] + text_shift
        y_ = coordinates[idx,1] + text_shift
        text = idx_to_text[idx]
        mtext(p, x_, y_, text, text_color, text_font_size, text_alpha)

def draw(coordinates, idx_to_text, p=None, title=None, marker="circle",
         marker_color='gray', marker_size=5, marker_alpha=0.5):
    # prepare figure
    if p is None:
        p = initialize_figure(title)

    # prepare data source
    source = ColumnDataSource(data=dict(
        x = coordinates[:,0].tolist(),
        y = coordinates[:,1].tolist(),
        desc = idx_to_text
    ))

    # scatter plot
    p.scatter('x', 'y', marker=marker, size=marker_size, line_color= 'white',
              fill_color= marker_color, alpha=marker_alpha, source=source)
    return p

p = draw(z_wv, idx_to_vocab, marker_size=6, marker_alpha=0.3, marker_color='gray',
         title='Joint visualization of words and docs t_sne')
p = draw(z_dv, idx_to_doc, p=p, marker_size=6, marker_alpha=0.8, marker_color='blue')
p = draw(z_av, artist, p=p, marker_size=7, marker_alpha=1, marker_color='red')

reset_output()
output_notebook()
output_file('joint_visualization_window=10_tsne.html')

p.height = 1000
p.width = 1500
show(p)

Loading BokehJS ...

In [151]:
total.head()

,key,id,x,y,Group,attritube
0,Artist_김정태,김정태,-25.856247,4.081685,artist,"{'id': '김정태', 'project': ['팀잇', '어디가 프로젝트'], '..."
1,Artist_박승순,박승순,12.920586,6.756471,artist,"{'id': '박승순', 'project': ['모멘텀', '인터미디어 모빌리티 R..."
2,Artist_양아치,양아치,20.153425,-16.116323,artist,"{'id': '양아치', 'project': ['Screen Screen Scree..."
3,Artist_이장원,이장원,35.738045,-15.674254,artist,"{'id': '이장원', 'project': ['아르키메데스 구멍 - 149,597..."
4,Artist_조호영,조호영,22.023046,13.988201,artist,"{'id': '조호영', 'project': ['랜드스케이프데이터프로젝트', '엠알..."


In [152]:
len(total.index)

1154

In [153]:
df[df['프로젝트명'] == '팀잇'][['프로젝트명','작품명','년도','작가명','키워드','내부파트너','외부파트너','매체']]

,프로젝트명,작품명,년도,작가명,키워드,내부파트너,외부파트너,매체
0,팀잇,<플레이팅해보기>,2018,김정태,#디지털 소재 #픽셀 #변형 #VR아트,"김정태, 최영준",NaN,영상 및 텍스트
